In [49]:
import tensorflow as tf
import numpy as np
from json import load
import spacy
import nltk
import string
from nltk.corpus import stopwords
from string import punctuation
from spacy.matcher import PhraseMatcher
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, mean_squared_error
from sklearn.preprocessing import normalize
nlp = spacy.load("ru_core_news_sm")


# Make dataset

In [50]:
with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discription_of_event = load(js)


In [51]:
kcm_num = {'знать': 1, 'уметь': 2, 'владеть': 3}


In [52]:
kcm = {'знать': [], 'уметь': [], 'владеть': []}
for i in discription_of_event.values():
    for j in i:
        for z in i[j]:
            kcm[j].append(list(nlp(z).vector))


In [53]:
k = []
for i in kcm:
    for j in kcm[i]:
        k.append(j+[i])


In [54]:
data = pd.DataFrame(k, columns=range(len(k[0])))
data


,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,0.971926,-0.246689,-0.183886,-0.137504,0.702932,-0.080612,2.031338,0.200807,-0.231049,-0.340155,...,0.665688,1.202340,0.664174,1.212046,0.472954,0.429120,1.139365,0.221086,0.465204,знать
1,0.911910,0.018401,-0.091500,-0.094755,0.738385,0.123497,2.089792,-0.142038,-0.317070,-0.059940,...,0.743450,0.956466,0.645719,1.061606,0.343979,0.329694,1.237762,0.190863,0.420430,знать
2,1.021014,0.184183,0.433282,-0.278907,0.801618,-0.852197,0.665390,0.474348,-0.338191,0.011781,...,0.578268,1.186024,0.373206,-0.105724,0.697767,1.503142,0.473007,0.019525,0.105892,знать
3,1.007172,1.746930,-0.499345,0.154830,0.095761,-0.146930,-1.046818,0.095326,-1.211388,-1.536004,...,0.010126,0.197326,1.649822,-0.225561,0.970883,1.863847,1.765509,-0.225985,0.810576,знать
4,0.782499,1.892098,-1.024817,-0.116163,0.046335,-0.256544,-1.027776,0.389015,-1.398157,-1.311625,...,-0.066063,0.487484,1.715499,-1.021188,1.147734,1.297385,1.459508,-0.899512,0.568933,знать
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,0.825537,-0.485574,-0.570523,0.342539,0.612211,-0.413298,1.359276,2.487679,-0.167546,-0.549826,...,0.462875,0.679714,0.490503,0.296399,0.293366,0.018967,1.103019,-0.813560,0.292161,владеть
703,0.794440,-0.472995,-0.540337,0.378898,0.568074,-0.344468,1.367658,2.408878,-0.102106,-0.605873,...,0.479785,0.663571,0.436278,0.311820,0.296010,-0.046259,1.114526,-0.899259,0.306704,владеть
704,0.816133,-0.522040,-0.578947,0.383433,0.565011,-0.473453,1.345864,2.455451,-0.181931,-0.582975,...,0.513654,0.651401,0.410415,0.341482,0.390819,-0.032848,1.037540,-0.830624,0.259147,владеть
705,1.391641,-0.522611,-0.744685,0.839048,0.592009,-0.398238,1.133930,2.097048,0.112283,-0.640256,...,0.037263,0.397545,0.307692,-0.197188,0.826490,0.327574,0.952950,-0.733257,0.058681,владеть


In [55]:
data_num = data.drop(96, axis=1).join(data[96].map(lambda a: kcm_num[a]))
data_num


,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,0.971926,-0.246689,-0.183886,-0.137504,0.702932,-0.080612,2.031338,0.200807,-0.231049,-0.340155,...,0.665688,1.202340,0.664174,1.212046,0.472954,0.429120,1.139365,0.221086,0.465204,1
1,0.911910,0.018401,-0.091500,-0.094755,0.738385,0.123497,2.089792,-0.142038,-0.317070,-0.059940,...,0.743450,0.956466,0.645719,1.061606,0.343979,0.329694,1.237762,0.190863,0.420430,1
2,1.021014,0.184183,0.433282,-0.278907,0.801618,-0.852197,0.665390,0.474348,-0.338191,0.011781,...,0.578268,1.186024,0.373206,-0.105724,0.697767,1.503142,0.473007,0.019525,0.105892,1
3,1.007172,1.746930,-0.499345,0.154830,0.095761,-0.146930,-1.046818,0.095326,-1.211388,-1.536004,...,0.010126,0.197326,1.649822,-0.225561,0.970883,1.863847,1.765509,-0.225985,0.810576,1
4,0.782499,1.892098,-1.024817,-0.116163,0.046335,-0.256544,-1.027776,0.389015,-1.398157,-1.311625,...,-0.066063,0.487484,1.715499,-1.021188,1.147734,1.297385,1.459508,-0.899512,0.568933,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,0.825537,-0.485574,-0.570523,0.342539,0.612211,-0.413298,1.359276,2.487679,-0.167546,-0.549826,...,0.462875,0.679714,0.490503,0.296399,0.293366,0.018967,1.103019,-0.813560,0.292161,3
703,0.794440,-0.472995,-0.540337,0.378898,0.568074,-0.344468,1.367658,2.408878,-0.102106,-0.605873,...,0.479785,0.663571,0.436278,0.311820,0.296010,-0.046259,1.114526,-0.899259,0.306704,3
704,0.816133,-0.522040,-0.578947,0.383433,0.565011,-0.473453,1.345864,2.455451,-0.181931,-0.582975,...,0.513654,0.651401,0.410415,0.341482,0.390819,-0.032848,1.037540,-0.830624,0.259147,3
705,1.391641,-0.522611,-0.744685,0.839048,0.592009,-0.398238,1.133930,2.097048,0.112283,-0.640256,...,0.037263,0.397545,0.307692,-0.197188,0.826490,0.327574,0.952950,-0.733257,0.058681,3


In [56]:
data_OH = data.drop(96, axis=1).join(pd.get_dummies(data[96]))
data_OH


,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,владеть,знать,уметь
0,0.971926,-0.246689,-0.183886,-0.137504,0.702932,-0.080612,2.031338,0.200807,-0.231049,-0.340155,...,0.664174,1.212046,0.472954,0.429120,1.139365,0.221086,0.465204,0,1,0
1,0.911910,0.018401,-0.091500,-0.094755,0.738385,0.123497,2.089792,-0.142038,-0.317070,-0.059940,...,0.645719,1.061606,0.343979,0.329694,1.237762,0.190863,0.420430,0,1,0
2,1.021014,0.184183,0.433282,-0.278907,0.801618,-0.852197,0.665390,0.474348,-0.338191,0.011781,...,0.373206,-0.105724,0.697767,1.503142,0.473007,0.019525,0.105892,0,1,0
3,1.007172,1.746930,-0.499345,0.154830,0.095761,-0.146930,-1.046818,0.095326,-1.211388,-1.536004,...,1.649822,-0.225561,0.970883,1.863847,1.765509,-0.225985,0.810576,0,1,0
4,0.782499,1.892098,-1.024817,-0.116163,0.046335,-0.256544,-1.027776,0.389015,-1.398157,-1.311625,...,1.715499,-1.021188,1.147734,1.297385,1.459508,-0.899512,0.568933,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,0.825537,-0.485574,-0.570523,0.342539,0.612211,-0.413298,1.359276,2.487679,-0.167546,-0.549826,...,0.490503,0.296399,0.293366,0.018967,1.103019,-0.813560,0.292161,1,0,0
703,0.794440,-0.472995,-0.540337,0.378898,0.568074,-0.344468,1.367658,2.408878,-0.102106,-0.605873,...,0.436278,0.311820,0.296010,-0.046259,1.114526,-0.899259,0.306704,1,0,0
704,0.816133,-0.522040,-0.578947,0.383433,0.565011,-0.473453,1.345864,2.455451,-0.181931,-0.582975,...,0.410415,0.341482,0.390819,-0.032848,1.037540,-0.830624,0.259147,1,0,0
705,1.391641,-0.522611,-0.744685,0.839048,0.592009,-0.398238,1.133930,2.097048,0.112283,-0.640256,...,0.307692,-0.197188,0.826490,0.327574,0.952950,-0.733257,0.058681,1,0,0


# Classifier

In [57]:
X_train_num, X_test_num, y_train_num, y_test_num = train_test_split(
    data.drop(96, axis=1), data_num[96], test_size=0.1, random_state=42)


In [58]:
import tensorflow as tf

model_class = tf.keras.models.Sequential()
model_class.add(tf.keras.layers.Dense(56, activation='relu',
                                input_shape=(96,), name='hidden_layer_1'))
model_class.add(tf.keras.layers.Dense(
    56, activation='relu', name='hidden_layer_2'))
model_class.add(tf.keras.layers.Dense(
    16, activation='relu', name='hidden_layer_3'))
model_class.add(tf.keras.layers.Dense(
    8, activation='relu', name='hidden_layer_4'))
model_class.add(tf.keras.layers.Dense(1, name='output', activation='sigmoid'))
model_class.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer_1 (Dense)      (None, 56)                5432      
                                                                 
 hidden_layer_2 (Dense)      (None, 56)                3192      
                                                                 
 hidden_layer_3 (Dense)      (None, 16)                912       
                                                                 
 hidden_layer_4 (Dense)      (None, 8)                 136       
                                                                 
 output (Dense)              (None, 1)                 9         
                                                                 
Total params: 9,681
Trainable params: 9,681
Non-trainable params: 0
_________________________________________________________________


In [59]:
# model_class = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(5, input_shape=(10,), name='hidden_layer_1'),
#     tf.keras.layers.Dense(2, name='output')
# ])

model_class.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True),  # Функция потерь
    optimizer='Adam',  # Оптимизатор
    metrics=[  # Метрики
        'accuracy',  # Если у объекта назначено имя, то можно вызвать объект с его помощью
        tf.keras.metrics.Precision()
    ]
)


In [60]:
# model_class = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(1, input_shape=(96,)),
#     tf.keras.layers.Dense(1,)
# ])

# Скомпилируем
model_class.compile(
    optimizer='Adam',
    loss='mse',
    metrics=['mean_absolute_error']
)

# Обучим
model_class.fit(
    X_train_num,  # Набор входных данных
    y_train_num,  # Набор правильных ответов
    validation_split=0.2,  # Этот параметр автоматически выделит часть обучающего набора на валидационные данные. В данном случа 20%
    epochs=10,  # Процесс обучения завершится после 10 эпох
    # Набор данных будет разбит на пакеты (батчи) по 8 элементов набора в каждом.
    batch_size=8
)


Epoch 1/10
64/64 [==============================] - 1s 3ms/step - loss: 2.0704 - mean_absolute_error: 1.1907 - val_loss: 1.3699 - val_mean_absolute_error: 0.9000
Epoch 2/10
64/64 [==============================] - 0s 2ms/step - loss: 1.5704 - mean_absolute_error: 0.9791 - val_loss: 1.3675 - val_mean_absolute_error: 0.8987
Epoch 3/10
64/64 [==============================] - 0s 2ms/step - loss: 1.5694 - mean_absolute_error: 0.9786 - val_loss: 1.3673 - val_mean_absolute_error: 0.8985
Epoch 4/10
64/64 [==============================] - 0s 2ms/step - loss: 1.5691 - mean_absolute_error: 0.9785 - val_loss: 1.3673 - val_mean_absolute_error: 0.8985
Epoch 5/10
64/64 [==============================] - 0s 2ms/step - loss: 1.5690 - mean_absolute_error: 0.9784 - val_loss: 1.3672 - val_mean_absolute_error: 0.8985
Epoch 6/10
64/64 [==============================] - 0s 2ms/step - loss: 1.5690 - mean_absolute_error: 0.9784 - val_loss: 1.3672 - val_mean_absolute_error: 0.8985
Epoch 7/10
64/64 [==========

In [61]:
predicted_num = model_class.predict(X_test_num)


3/3 [==============================] - 0s 2ms/step


In [62]:
predicted_num.shape, y_test_num.to_numpy().shape


((71, 1), (71,))

In [63]:
predicted_num, y_test_num.to_numpy()


(array([[1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [0.9999992 ],
        [1.        ],
        [0.99999994],
        [1.        ],
        [1.        ],
        [1.        ],
        [0.9999999 ],
        [1.        ],
        [0.9998906 ],
        [1.        ],
        [0.9999999 ],
        [1.        ],
        [1.        ],
        [1.        ],
        [0.9999975 ],
        [0.9999993 ],
        [1.        ],
        [1.        ],
        [0.9999999 ],
        [1.        ],
        [0.99997604],
        [1.        ],
        [1.        ],
        [1.        ],
        [0.9999999 ],
        [1.        ],
        [0.9999814 ],
        [0.9999986 ],
        [1.        ],
        [0.99998367],
        [1.        ],
        [1.        ],
        [0.9999997 ],
        [1.        ],
        [0.9999996 ],
        [1.        ],
        [1.        ],
        [0.9999997 ],
        [1.        ],
        [0.99999994],
        [0.99999994],
        [1

In [64]:
model_class.predict(np.array([nlp('программировать на python').vector]))
# nlp('программировать на python').vector


1/1 [==============================] - 0s 18ms/step


array([[0.99994767]], dtype=float32)

# Coder

In [65]:
X_train_OH, X_test_OH, y_train_OH, y_test_OH = train_test_split(
    data.drop(96, axis=1), data_OH.loc[:, ['владеть', 'знать', 'уметь']], test_size=0.25, random_state=42)


In [66]:
import tensorflow as tf

model_coder = tf.keras.models.Sequential()
model_coder.add(tf.keras.layers.Dense(56, activation='relu',
                                input_shape=(96,), name='hidden_layer_1'))
model_coder.add(tf.keras.layers.Dense(
    56, activation='relu', name='hidden_layer_2'))
model_coder.add(tf.keras.layers.Dense(
    16, activation='relu', name='hidden_layer_3'))
model_coder.add(tf.keras.layers.Dense(
    8, activation='relu', name='hidden_layer_4'))
model_coder.add(tf.keras.layers.Dense(3, name='output'))
model_coder.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer_1 (Dense)      (None, 56)                5432      
                                                                 
 hidden_layer_2 (Dense)      (None, 56)                3192      
                                                                 
 hidden_layer_3 (Dense)      (None, 16)                912       
                                                                 
 hidden_layer_4 (Dense)      (None, 8)                 136       
                                                                 
 output (Dense)              (None, 3)                 27        
                                                                 
Total params: 9,699
Trainable params: 9,699
Non-trainable params: 0
_________________________________________________________________


In [67]:
# model_coder = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(3, input_shape=(96,)),
#     tf.keras.layers.Dense(3,)
# ])

# Скомпилируем
model_coder.compile(
    optimizer='Adam',
    loss='mse',
    metrics=['mse']
)

# Обучим
model_coder.fit(
    X_train_OH,  # Набор входных данных
    y_train_OH,  # Набор правильных ответов
    validation_split=0.2,  # Этот параметр автоматически выделит часть обучающего набора на валидационные данные. В данном случа 20%
    epochs=20,  # Процесс обучения завершится после 10 эпох
    # Набор данных будет разбит на пакеты (батчи) по 8 элементов набора в каждом.
    batch_size=3
)


Epoch 1/20
142/142 [==============================] - 1s 2ms/step - loss: 0.3125 - mse: 0.3125 - val_loss: 0.2475 - val_mse: 0.2475
Epoch 2/20
142/142 [==============================] - 0s 2ms/step - loss: 0.1812 - mse: 0.1812 - val_loss: 0.0985 - val_mse: 0.0985
Epoch 3/20
142/142 [==============================] - 0s 2ms/step - loss: 0.0788 - mse: 0.0788 - val_loss: 0.0401 - val_mse: 0.0401
Epoch 4/20
142/142 [==============================] - 0s 2ms/step - loss: 0.0324 - mse: 0.0324 - val_loss: 0.0239 - val_mse: 0.0239
Epoch 5/20
142/142 [==============================] - 0s 2ms/step - loss: 0.0174 - mse: 0.0174 - val_loss: 0.0197 - val_mse: 0.0197
Epoch 6/20
142/142 [==============================] - 0s 2ms/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.0193 - val_mse: 0.0193
Epoch 7/20
142/142 [==============================] - 0s 2ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.0169 - val_mse: 0.0169
Epoch 8/20
142/142 [==============================] - 0s 2ms/step - loss: 0.

In [68]:
predicted_OH = model_coder.predict(X_test_OH)


6/6 [==============================] - 0s 1ms/step


In [69]:
predicted_OH.shape, y_test_OH.to_numpy().shape


((177, 3), (177, 3))

In [70]:
mean_squared_error(predicted_OH.T[0], y_test_OH.to_numpy().T[0]), mean_squared_error(predicted_OH.T[1], y_test_OH.to_numpy().T[1]), mean_squared_error(predicted_OH.T[2], y_test_OH.to_numpy().T[2])


(0.017738715, 0.0125685455, 0.012998964)

In [71]:
f = lambda a: f'{a}: {normalize(model_coder.predict(np.array([nlp(a).vector])))}'
print(
    'владеть знать уметь',
    f('программировать на python'),
    f('определить степень вероятности кризиса в компании'),
    f('Какие показатели хозяйственной деятельности предприятия необходимо анализировать'),
    sep='\n'
    )


1/1 [==============================] - 0s 17ms/step
владеть знать уметь
программировать на python: [[-0.12386788 -0.03150187  0.9917985 ]]
определить степень вероятности кризиса в компании: [[0.10014762 0.06774728 0.9926635 ]]
Какие показатели хозяйственной деятельности предприятия необходимо анализировать: [[0.05392641 0.9418245  0.33175078]]


In [72]:
model_coder.save('../../PipeLines/Coder/coder_from_spacy_to_kcm_onh5.h5')